# Classifier Model

> A customisable pytorch classifier model.

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| default_exp Models.Classifier

In [ ]:
 #| export
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class Classifier(nn.Module):
    """A simple classifier model.
    """
    def __init__(self, input_size:int, # number of features
                  hidden_sizes:list , output_size, dropout, use_batch_norm, spectral_norm=False):
        super().__init__()

        self.input_size = input_size
        self.dropout = dropout
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.use_batch_norm = use_batch_norm
        self.spectral_norm = spectral_norm
            
        if spectral_norm:
            self.spectral_norm = nn.utils.spectral_norm

        # create a list of layers
        layers = []

        # input layer
        if spectral_norm:
            layers.append(self.spectral_norm(nn.Linear(self.input_size, self.hidden_sizes[0])))
        else:
            layers.append(nn.Linear(self.input_size, self.hidden_sizes[0]))
        layers.append(nn.LeakyReLU(0.2))
        if self.dropout > 0:
            layers.append(nn.Dropout(p=self.dropout))

        # hidden layers
        for i in range(1, len(self.hidden_sizes)):
            if spectral_norm:
                layers.append(self.spectral_norm(nn.Linear(self.hidden_sizes[i-1], self.hidden_sizes[i])))
            else:
                layers.append(nn.Linear(self.hidden_sizes[i-1], self.hidden_sizes[i]))
            if self.use_batch_norm:
                layers.append(nn.InstanceNorm1d(self.hidden_sizes[i]))
            layers.append(nn.LeakyReLU(0.2))
            if self.dropout > 0:
                layers.append(nn.Dropout(p=self.dropout))

        # output layer
        layers.append(nn.Linear(self.hidden_sizes[-1], self.output_size))
        # create the model using Sequential
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        x = self.model(x)
        return F.softmax(x,dim=1)
      


In [ ]:
from sc_generative_demo.Models.Classifier import *

In [ ]:
classifier = Classifier(10, [20, 30], 2, 0.1, True, spectral_norm=False)

In [ ]:
classifier

Classifier(
  (model): Sequential(
    (0): Linear(in_features=10, out_features=20, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=20, out_features=30, bias=True)
    (4): InstanceNorm1d(30, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (5): LeakyReLU(negative_slope=0.2)
    (6): Dropout(p=0.1, inplace=False)
    (7): Linear(in_features=30, out_features=2, bias=True)
  )
)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()